# 1. Setup

IMPORTANT!! You must be connected to a GPU or TPU in order to be able to load the Huggingface model.

# 2. Dependencies

In [ ]:
#module2

!pip install llama-cpp-python==0.1.78

In [ ]:
# module 2
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 --force-reinstall --upgrade --no-cache-dir --verbose

In [ ]:
!pip install streamlit

In [4]:
! wget -w -O - ipv4.icanhazip.com

wget: --wait: Invalid time period ‘-O’


# 3. Modules

In [5]:
%%writefile load_hf.py
from huggingface_hub import login # notebook_login

def load_huggingface_accesstoken(token):
  """
  This function loads huggingface access token so you can get access to models such as Llama2
  """
  try:
    login(token=token)
    print("Successfully logged in to Hugging Face")

  except Exception as e:
    print(f"Failed to log in to Hugging Face: {e}")

Writing load_hf.py


In [6]:
%%writefile module1.py
import pandas as pd
import streamlit as st

def read_csv(csv_file):
    """
    This function reads a .csv file, confirms it is a valid Citeasyon format, and returns a dataframe.
    Any column not found in the expected columns will be removed.
    """

    expected_citeasyon_cols = ["file_id", "title", "author_name", "abstract", "keywords",
                               "journal", "volume", "proceedings", "year", "month", "doi",
                               "url", "pages", "field", "topic"]

    try:
        # read .csv, automatically detect and remove any index column like 'Unnamed: 0'
        df = pd.read_csv(csv_file)

        # normalize column names in both lists
        df.columns = [col.strip().lower() for col in df.columns]
        expected_columns = [col.strip().lower() for col in expected_citeasyon_cols]

        # keep only the columns that are in the expected columns list
        df = df[[col for col in df.columns if col in expected_columns]]

        # if the remaining columns match the expected columns
        if set(df.columns) == set(expected_columns):
            st.success("CSV file is valid Citeasyon FORMAT")
            return df
        else:
            st.error(f"CSV file is not valid Citeasyon FORMAT. Expected columns: {expected_citeasyon_cols}")
            st.error(f"Found columns: {df.columns.tolist()}")
            return None

    except Exception as e:
        st.error(f"Error reading CSV file: {e}")
        return None


# upload and process the .csv file
def load_data():
    uploaded_file = st.file_uploader("Select your Citeasyon DataBase", type=["csv"], accept_multiple_files=False)

    if uploaded_file is not None:
        try:
            input_database = read_csv(uploaded_file)
            if input_database is not None:
                st.write(input_database)
                return input_database
            else:
                st.warning("No valid data to process.")
        except Exception as e:
            st.error(f"Error processing file: {e}")
            return None
    else:
        st.info("Please upload a CSV file.")
        return None



Writing module1.py


In [32]:
# module 2 version 3

%%writefile module2.py

import pandas as pd
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
import torch

def load_model():
    model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
    model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"  # model in bin format
    return model_name_or_path, model_basename

def model_gpu(model_path):
    # Initialize model with optimized parameters for memory management
    lcpp_llm = Llama(
        model_path=model_path,
        n_threads=2,  # Use 2 CPU cores
        n_batch=128,  # reduced batch size for lower memory usage
        n_gpu_layers=20,  # adjusted GPU layers
        n_ctx=2048,  # reduced context window to fit within memory
    )
    return lcpp_llm



def llm_citation_assistant(citation_type, gpu_for_model, database):
    dictionary_citations = {}

    # iterate over each row in the database
    for index, row in database.iterrows():
        # construct the prompt from row information
        information_article = f"""[s]Title: {row["title"]}[/s] [s]Authors: {row["author_name"]}[/s] [s]Journal: {row["journal"]}[/s] [s]Proceedings: {row["proceedings"]}[/s] [s]Volume: {row["volume"]}[/s] [s]Year: {row["year"]}[/s] [s]DOI: {row["doi"]}[/s] [s]Pages: {row["pages"]}[/s]"""
        full_prompt = information_article

        prompt_template = f'''
SYSTEM: You are in charge of formatting citations into the latest version known of {citation_type} style. Your output is the final citations version. If there is an element missing, you put an informative placeholder like TITLE if title is missing. Return the citation directly, no explanation.
USER: ARTICLE INFORMATION: {full_prompt}
ASSISTANT:'''

        try:
            response = gpu_for_model(
                prompt=prompt_template,
                max_tokens=256,
                temperature=0.1,
                top_p=0.95,
                repeat_penalty=1.2,
                top_k=50,
                stop=['USER:', 'SYSTEM:', 'ASSISTANT:'],
                echo=False  # Do not echo the prompt back
            )

            # extract response from the models answer
            assistant_response = response["choices"][0]["text"].strip()

            # make sure the response is cleaned up and only contains the citation
            assistant_response = assistant_response.split('ASSISTANT:')[-1].strip()

            # store results in dictionary format
            dictionary_citations[row["file_id"]] = assistant_response

        except Exception as e:
            print(f"Error processing row {index} ({row['file_id']}): {e}")
            dictionary_citations[row["file_id"]] = "Error"

    return dictionary_citations





def citation_generator(citation_type, database):
    model_name_or_path, model_basename = load_model()

    # Download model and log path
    try:
        model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)
        print(f"Model downloaded to: {model_path}")
    except Exception as e:
        print(f"Failed to download model: {e}")
        return

    # Load model to GPU
    try:
        gpu_for_model = model_gpu(model_path=model_path)
    except Exception as e:
        print(f"Failed to load model to GPU: {e}")
        return

    print("Module 2: citation_generator started")

    dictionary_citations = {}

    # Extract information and update citations
    if citation_type:
        dictionary_citations = llm_citation_assistant(citation_type, gpu_for_model, database)
        print(dictionary_citations)

    else:
        print("Invalid citation type or not implemented yet")

    return dictionary_citations


Overwriting module2.py


# 4. app.py

In [33]:
%%writefile app.py

# version 3

import streamlit as st
import pandas as pd

from load_hf import load_huggingface_accesstoken
from module1 import load_data
from module2 import citation_generator

st.set_page_config(page_title='Citeasyon', page_icon=":tada:", layout="wide")

# HEADER
with st.container():
    st.title(":red[_CITEASYON step 2_] :sparkles:")


def run_citeasyon_step2():
    ## 0. hugging face access token
    token = st.text_input("Enter your Hugging Face access Token", type="password")
    if token:
        load_huggingface_accesstoken(token)
    else:
        st.warning("Access token for Hugging Face required")
        return  # Exit if token is not provided

    ## 1. load data
    input_database = load_data()

    # choose MLA or APA citation style
    st.sidebar.header("Citation Format")
    citation_options = st.sidebar.multiselect("Select:", ["MLA", "APA"])

    # check if the user has selected at least one citation style
    if not citation_options:
        st.warning("Please select at least one citation format.")
        return  # exit if no citation format is selected

    # "Go" button to start the citation
    if st.sidebar.button("Go"):
        # init citations dictionary in session state
        if 'citations_dictionary' not in st.session_state:
            st.session_state.citations_dictionary = {}

        for citation_type in citation_options:
            cites_complete = citation_generator(citation_type, input_database)

            # update session state dictionary with new citations
            st.session_state.citations_dictionary.update(cites_complete)

            st.write(f"Generated Citations for {citation_type}:")
            for cite in cites_complete.values():
                st.write(cite)

            citations_text = "\n".join(cites_complete.values())
            st.download_button(
                label='Download citations',
                data=citations_text,
                file_name=f'{citation_type}_citeasyons.txt',
                mime='text/plain'
            )

    else:
        st.info("Click 'Go' to start the citation generation process.")

run_citeasyon_step2()


Overwriting app.py


# 5. Run local tunnel

In [ ]:
! streamlit run app.py & npx localtunnel --port 8501
